In [4]:
!pip install sentence-transformers nltk textstat pandas matplotlib wordcloud --quiet

# Imports
import nltk
import re
from sentence_transformers import SentenceTransformer
import pandas as pd
import ast
import tokenize
import io
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# Download NLTK data, including punkt_tab
try:
    nltk.download('punkt_tab', quiet=True)
    nltk.download('stopwords', quiet=True)
except Exception as e:
    print(f"Error downloading NLTK data: {e}")
print("[✓] Setup complete")

[✓] Setup complete


In [5]:
Master_codes = [
    """1. Swap Two Variables Without a Temporary Variable
a = 5
b = 10
a, b = b, a
print("After swapping: a =", a, "b =", b)
""",


"""2. Check if a Number is Even or Odd
num = int(input("Enter a number: "))
if num % 2 == 0:
    print(num, "is even")
else:
    print(num, "is odd")
    """,



""" 3. Leap Year Checker
year = int(input("Enter a year: "))
if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
    print(year, "is a leap year")
else:
    print(year, "is not a leap year")
    """,



""" 4. Print Multiplication Table
num = int(input("Enter a number: "))
for i in range(1, 11):
    print(num, "x", i, "=", num * i)
    """,



"""5. Palindrome Checker
text = input("Enter a word: ")
if text == text[::-1]:
    print("It's a palindrome")
else:
    print("Not a palindrome")""",

"""6. Find the Largest of Three Numbers
a = int(input("Enter first number: "))
b = int(input("Enter second number: "))
c = int(input("Enter third number: "))
largest = max(a, b, c)
print("The largest number is:", largest)
""",



""" 7. Calculate Simple Interest
principal = float(input("Enter principal amount: "))
rate = float(input("Enter annual interest rate (%): "))
time = float(input("Enter time in years: "))
simple_interest = (principal * rate * time) / 100
print("Simple Interest is:", simple_interest)
""",



""" 8. Count Words in a Sentence
sentence = input("Enter a sentence: ")
words = sentence.split()
print("Number of words:", len(words))
""",



"""9. Check if a Number is Armstrong
num = int(input("Enter a number: "))
sum = 0
temp = num
while temp > 0:
    digit = temp % 10
    sum += digit ** 3
    temp //= 10
if num == sum:
    print(num, "is an Armstrong number")
else:
    print(num, "is not an Armstrong number")
    """,



"""10. Convert Celsius to Fahrenheit
celsius = float(input("Enter temperature in Celsius: "))
fahrenheit = (celsius * 9/5) + 32
print("Temperature in Fahrenheit:", fahrenheit)
"""
]

Master_names = [f"Program_{i+1}" for i in range(len(Master_codes))]

print("Top 10 Example of Code Masters:")
for name, text in zip(Master_names, Master_codes):
    print(f"{name}: {len(text)} characters")








Top 10 Example of Code Masters:
Program_1: 118 characters
Program_2: 151 characters
Program_3: 201 characters
Program_4: 132 characters
Program_5: 142 characters
Program_6: 219 characters
Program_7: 280 characters
Program_8: 132 characters
Program_9: 277 characters
Program_10: 173 characters


In [6]:
class CodeExplainer:
    def __init__(self):
        self.models = {
            'MiniLM': SentenceTransformer('all-MiniLM-L12-v2'),
            'DistilRoBERTa': SentenceTransformer('all-distilroberta-v1'),
            'MPNet': SentenceTransformer('all-mpnet-base-v2')
        }
        self.explanation_templates = {
            'def': [
                "Defines function '{}' with parameters",  # MiniLM
                "Creates a function '{}' for computation",  # DistilRoBERTa
                "Sets up function '{}' to perform a task"  # MPNet
            ],
            'if': [
                "Evaluates condition: '{}'",
                "Checks if: '{}'",
                "Tests condition: '{}'"
            ],
            'elif': [
                "Evaluates additional condition: '{}'",
                "Checks another condition: '{}'",
                "Tests further condition: '{}'"
            ],
            'else': [
                "Handles alternative case when conditions fail",
                "Executes if prior conditions are false",
                "Catches remaining cases"
            ],
            'for': [
                "Loops over sequence: '{}'",
                "Iterates through: '{}'",
                "Performs iteration: '{}'"
            ],
            'while': [
                "Loops while: '{}'",
                "Continues looping if: '{}'",
                "Repeats until condition changes: '{}'"
            ],
            'assign': [
                "Assigns value to '{}'",
                "Sets variable '{}' to a value",
                "Stores result in '{}'"
            ],
            'print': [
                "Outputs: '{}'",
                "Displays to console: '{}'",
                "Prints result: '{}'"
            ],
            'return': [
                "Returns: '{}'",
                "Yields value: '{}'",
                "Provides output: '{}'"
            ],
            'default': [
                "Executes: '{}'",
                "Performs operation: '{}'",
                "Runs code: '{}'"
            ]
        }

    def explain_line(self, line, model_name):
        line = line.strip()
        if not line or line.startswith('#'):
            return None
        # Mock semantic analysis: Use model index to select explanation variant
        model_index = {'MiniLM': 0, 'DistilRoBERTa': 1, 'MPNet': 2}
        idx = model_index[model_name]
        # Compute embedding for context (mock usage)
        self.models[model_name].encode([line])[0]
        if line.startswith('def '):
            return self.explanation_templates['def'][idx].format(line.split('(')[0].replace('def ', ''))
        elif line.startswith('if '):
            return self.explanation_templates['if'][idx].format(line)
        elif line.startswith('elif '):
            return self.explanation_templates['elif'][idx].format(line)
        elif line.startswith('else:'):
            return self.explanation_templates['else'][idx]
        elif line.startswith('for '):
            return self.explanation_templates['for'][idx].format(line)
        elif line.startswith('while '):
            return self.explanation_templates['while'][idx].format(line)
        elif '=' in line and not line.startswith('print'):
            return self.explanation_templates['assign'][idx].format(line.split('=')[0].strip())
        elif line.startswith('print'):
            return self.explanation_templates['print'][idx].format(line)
        elif line.startswith('return'):
            return self.explanation_templates['return'][idx].format(line)
        return self.explanation_templates['default'][idx].format(line)

    def explain_code(self, code_text, model_name):
        explanations = {}
        for i, line in enumerate(code_text.splitlines(), 1):
            explanation = self.explain_line(line, model_name)
            if explanation:
                explanations[i] = explanation
        return explanations

    def compare_explanations(self, code_text, snippet_name):
        # Generate explanations for the code snippet using all three models
        explanations = {
            model_name: self.explain_code(code_text, model_name)
            for model_name in self.models.keys()
        }
        # Create a DataFrame to compare explanations line by line
        comparison_data = []
        for line_num in sorted(set(sum([list(exp.keys()) for exp in explanations.values()], []))):
            row = {'Line Number': line_num, 'Code': code_text.splitlines()[line_num-1].strip()}
            for model_name in self.models.keys():
                row[model_name] = explanations[model_name].get(line_num, "No explanation")
            comparison_data.append(row)
        return pd.DataFrame(comparison_data)

explainer = CodeExplainer()

# Generate and compare explanations for all snippets
comparisons = []
for name, text in zip(Master_codes ,Master_names ):
    comparison_df = explainer.compare_explanations(text, name)
    comparisons.append((name, comparison_df))

# Print comparison for Program 1
print("\nExplanation Comparison for Program_1:")
print(comparisons[0][1].to_string(index=False))

# Print summary of differences for all snippets
print("\nSummary of Explanation Differences:")
for name, df in comparisons:
    print(f"\n{name}:")
    differing_lines = df[df['MiniLM'] != df['DistilRoBERTa']][['Line Number', 'Code', 'MiniLM', 'DistilRoBERTa', 'MPNet']]
    if not differing_lines.empty:
        print("Lines with differing explanations:")
        print(differing_lines.to_string(index=False))
    else:
        print("All models provided identical explanations.")


Explanation Comparison for Program_1:
 Line Number      Code                MiniLM                   DistilRoBERTa                  MPNet
           1 Program_1 Executes: 'Program_1' Performs operation: 'Program_1' Runs code: 'Program_1'

Summary of Explanation Differences:

1. Swap Two Variables Without a Temporary Variable
a = 5
b = 10
a, b = b, a
print("After swapping: a =", a, "b =", b)
:
Lines with differing explanations:
 Line Number      Code                MiniLM                   DistilRoBERTa                  MPNet
           1 Program_1 Executes: 'Program_1' Performs operation: 'Program_1' Runs code: 'Program_1'

2. Check if a Number is Even or Odd
num = int(input("Enter a number: "))
if num % 2 == 0:
    print(num, "is even")
else:
    print(num, "is odd")
    :
Lines with differing explanations:
 Line Number      Code                MiniLM                   DistilRoBERTa                  MPNet
           1 Program_2 Executes: 'Program_2' Performs operation: 'Program_2' Ru

# Task
Integrate the code comparison functionality into a Gradio interface.

## Install gradio

### Subtask:
Install the Gradio library.


**Reasoning**:
The subtask is to install the Gradio library. I will use pip to install it.



In [7]:
!pip install gradio --quiet

## Define gradio interface function

### Subtask:
Create a function that takes code text and model names as input and returns the comparison DataFrame.


**Reasoning**:
Define a function `get_code_comparison` that takes code text and model names, instantiates `CodeExplainer`, calls `compare_explanations`, and returns the resulting DataFrame.



In [8]:
def get_code_comparison(code_text, model_names):
    explainer = CodeExplainer()
    # We need to provide a snippet_name, but it's not used in compare_explanations logic
    comparison_df = explainer.compare_explanations(code_text, "User Code Snippet")
    # Filter the DataFrame to only include the specified model names
    if model_names:
        # Ensure 'Line Number' and 'Code' columns are always included
        cols_to_keep = ['Line Number', 'Code'] + model_names
        comparison_df = comparison_df[cols_to_keep]
    return comparison_df

## Create gradio interface

### Subtask:
Define the Gradio interface using the function created in the previous step.


**Reasoning**:
Define the Gradio interface using the `get_code_comparison` function and the specified inputs, outputs, title, and example.



In [9]:
import gradio as gr

model_choices = list(explainer.models.keys())

iface = gr.Interface(
    fn=get_code_comparison,
    inputs=[
        gr.Textbox(label="Enter Python Code", lines=20),
        gr.CheckboxGroup(choices=model_choices, label="Select Models to Compare", value=model_choices)
    ],
    outputs=gr.DataFrame(label="Explanation Comparison"),
    title="Code Explanation Comparison Tool",
    examples=[[Master_codes[0], model_choices]]
)

## Launch interface

### Subtask:
Launch the Gradio interface.


**Reasoning**:
Launch the Gradio interface so the user can interact with it.



In [10]:
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1e75e71e60f02f0c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Summary:

### Data Analysis Key Findings

*   The Gradio library was successfully installed using `pip install gradio --quiet`.
*   A Python function `get_code_comparison` was defined to take code text and a list of model names as input, and return a pandas DataFrame comparing the explanations from the selected models.
*   The Gradio interface `iface` was created using `gr.Interface`, connecting the `get_code_comparison` function to a textbox input for code, a checkbox group for model selection, and a DataFrame output to display the results.
*   The Gradio interface was successfully launched and made publicly accessible via a shareable URL.

### Insights or Next Steps

*   The created Gradio interface provides a user-friendly way to compare code explanations from different models side-by-side.
*   Further enhancements could include adding more models for comparison or allowing users to provide custom model outputs for comparison.
